In [ ]:
################################################################################################
# <토큰 임베딩 예시>
# sentence-bert와 Faiss 라이브러리를 이용하여 검색 MRR(Mean Reciprocal Rank)측정 예시 3
# - paragrahs 문단을 토큰별로 쪼개고, 쪼갠 토큰 임베딩벡터를 구해서 Faiss 인덱싱 생성(문장별 1개 Faiss 인덱싱 생성)
# - 쿼리문장 단어로 쪼개고, 쪼갠 토큰 임베딩벡터를 구해서 각 문단 Faiss 인덱스에서 토큰별 코사인유사도 합으로 스코어 계산.
# - 말뭉치는  KorQuAD_v1.0 파일과 aihub(aihub.or.kr) 뉴스 기사 기계독해 데이터 QuA 파일 3종류 이용
#
# 1. 검색모델 로딩
# 2. JSON 파일 로딩 후 df 만듬
# 3. 문장들을 단어로 분리
# 4. 문장 단어별 Faiss 임베딩 생성
# 5. 쿼리 후 단어 MAX 스코어 합 구함
# 6. MRR 계산
#
#-------------------------------------------------------------------------------
# 쿼리
# -단어일때 검색률 : 40%
# -문장일때 검샐률 : 68.2%
# -BM25=Mecab 적용시: 85%, 적용안할때: 50%
#-------------------------------------------------------------------------------
# sklenarn 으로 cosine 확인 예제
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_sim = cosine_similarity([embed_querys[0]], [embed_querys_1[0]]) # (1,768) 식에 2차원 배열입력되어야 함.
#-------------------------------------------------------------------------------
################################################################################################

import faiss
import numpy as np
import pandas as pd
import time
from tqdm.notebook import tqdm

from os import sys
sys.path.append('../../')
from myutils import GPU_info, seed_everything, mlogging, bi_encoder

logger = mlogging(loggername="MRR-VOCAB", logfilename="../../../log/MRR-VOCAB")
device = GPU_info()
#device = 'cpu'  # cpu 테스트 할때

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
seed = 111
query_num = 500             # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
search_k = 5                # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
faiss_index_method = 0      # 0= Cosine Similarity 적용(IndexFlatIP 사용), 1= Euclidean Distance 적용(IndexFlatL2 사용)
#------------------------------------------------------------------------------------
# param 인자 범위 체크
if faiss_index_method > 1 or faiss_index_method < 0:
    raise ValueError(f"faiss_index_method = {faiss_index_method} is not bad!!")
    
seed_everything(111)

In [ ]:
#-------------------------------------------------------------------------------------
# 1. 검색모델 로딩
# => bi_encoder 모델 로딩, polling_mode 설정
#-------------------------------------------------------------------------------------
from myutils import bi_encoder

bi_encoder_path = "bongsoo/kpf-sbert-v1.1"#"bongsoo/klue-sbert-v1" # kpf-sbert-v1.1 # klue-sbert-v1 # albert-small-kor-sbert-v1.1
pooling_mode = 'mean' # bert면=mean, albert면 = cls
out_dimension = 768    # 출력 임베딩 크기 지정 : 0=기본 모델 임베딩크기(768), 예:128=128 츨력임베딩 크기 

word_embedding_mode, bi_encoder = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, 
                                             pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
#bi_encoder.to(device)

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 2. JSON 파일 로딩 후 df 만듬
# => KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
#-------------------------------------------------------------------------------------------------------
from myutils import read_korquad_v1_json, read_aihub_qua_json

# list 데이터들을 dataframe으로 만듬
'''
contexts = [
        [1, '독도 독도 독도 해역 헬기 추락사고가 발생한 지 열하루가 지났지만 실종자 추가 발견 소식은 들려오지 않고 있다. 헬기 동체 잔해물과 부유물 등은 발견되고 있지만 정작 실종자들은 발견하지 못해 수색이 장기화될 것이라는 우려가 현실이 될 조짐을 보여 실종자 가족들의 애를 태우고 있다.범정부현장수습지원단(지원단)은 10일 오전 10시 브리핑에서 이날 오전까지 독도해역 수색 결과 4점의 부유물을 추가 발견, 인양했다고 밝혔다'], 
        [11,'대구시는 11일 내년 정부 예산 3조1330억원을 확보해 전년보다 2%(611억원) 늘어났다고 밝혔다.그러나 올해 국비 증액 규모(1817억원)와 비교해 절반 수준에 그치며, 물산업클러스터 r&d 사업(200억), 국립청소년진로직업체험 수련원 건립 등은 한푼도 반영되지 않았다.'],
        [21,'자유한국당 (이름) 의원이 "공수처는 정권 유지를 위한 수단으로, 공수처가 있었으면 조국 수사를 못했을 것"이라고 주장했다.최 의원은 18일 대구 호텔수성에서 열린 대구 경북중견언론인모임 <아시아포럼21> 토론회에서 "검찰은 정권 말기가 되면 정권에 칼을 들이댔다. 공수처는 검찰의 칼끝을 회피하기 위한 것"이라며 이같이 말했다'],
        [2,'27일 오전 천연기념물(201-2호)이며 멸종위기 2급인 큰고니 떼가 경북 포항시 북구 흥해읍 샛강에 날아들었다.이날 관측된 큰고니는 어미 8마리와 새끼 3마리다.앞서 지난 19일 큰고니 8마리가 올들어 처음으로 관측됐다.큰고니들은 포항시 흥해읍 샛강에서 수초 등을 먹고 휴식한 후 대구 안심습지와 낙동강 하구 등지로 날아갈 것으로 보인다'],
        [3,'3일 대구와 경북지역은 구름이 많겠으며, 동해안과 북부 내륙에는 비가 내릴 것으로 예상된다.대구기상청에 따르면 중국 북동지방에서 남하하는 고기압의 가장자리에 들어 구름이 많겠으며, 경북 북부 내륙은 북쪽을 지나는 약한 기압골의 영향을 받아 오후부터 비가 약하게 내리거나 빗방울이 떨어지겠다.'],
        [31,'각급 정부기관의 통신망을 관리할 국가정보자원관리원 대구센터가 31일 대구 동구 도학동에서 착공했다.4312억원을 투입해 2021년 8월 준공 예정인 대구센터는 86개 기관의 서비와 장비를 운용하게 된다.정부통합전산센터에서 명칭이 변경된 국가정보자원관리원은 현재 대전본원과 광주센터를 운영 중이다.대구센터는 급변하는 행정환경과 4차 산업혁명 시대에 맞춰 클라우드, 빅데이터 등 신기술이 접목된 지능형 전산센터로 구축된다'],
        [4,'26일 오후 11시44분쯤 대구 달서구 신당동의 한 아파트에서 원인 모를 불이 나 주민 1명이 다치고 수십명이 대피했다.27일 소방당국에 따르면 신고를 받고 소방차 24대와 소방대원 74명이 출동해 20분만에 불길을 잡았다.이 불로 아파트 2층에 사는 a씨(53)가 온 몸에 화상을 입어 병원으로 이송됐으며, 연기에 놀란 주민 50여명이 대피하는 소동을 빚었다.']
       ]
df_contexts = pd.DataFrame(contexts, columns=['contextid','context'])
print(df_contexts['context'].values)
'''

# KorQuAD_v1.0 혹은 aihub 뉴스 기사 기계독해 데이터 QuA 파일을 불러옴.
jsonfile = './data/KorQuAD_v1.0_dev.json' # VL_unanswerable.json # VL_text_entailment.json # VL_span_inference.json # KorQuAD_v1.0_dev.json
contexts, questions, answers, contextids, qcontextids = read_korquad_v1_json(jsonfile) # read_aihub_qua_json(jsonfile)

# list들을 zip 으로 묶고, dataframe 생성함
# context, contextid를 묶어서 context df 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])

# question, answer, contextids를 묶어서 question df 만듬
df_questions = pd.DataFrame((zip(questions, answers, qcontextids)), columns = ['question','answer', 'contextid'])

In [ ]:
#-------------------------------------------------------------------------------------
# 3. 문단 토큰별 Faiss 임베딩 생성
# => 각 문단 토큰별로 임베딩 생성후 Faiss에 문단별로 임베딩만들고 토큰 임베딩 값을 추가함
#-------------------------------------------------------------------------------------
from myutils import embed_text, fassi_index, divide_arr_avg

EMBED_DIVIDE_LEN=3 # 문단 임베딩 토큰을 몇개 토큰으로 나눠서 평균을 구할지 계수 설정

start = time.time() #시작 시간
faissindexlist = []

paragraphs = df_contexts.context.to_list()
token_embeds = embed_text(model=bi_encoder, paragraphs=paragraphs, token_embeddings=True, return_tensor=False)
print(f'*len:{len(token_embeds)}, *shape:{token_embeds[1].size()}')

for token_embed in tqdm(token_embeds):
    token_embed_arrs = token_embed.cpu().numpy().astype('float32')
    token_embed_divide_arrs = divide_arr_avg(embed_arr=token_embed_arr, divide_len=EMBED_DIVIDE_LEN)# EMBED_DIVIDE_LEN 만큼 자르면서 문단 토큰 평균을 구함
    index = fassi_index(embeddings=token_embed_divide_arrs, method=faiss_index_method)
    faissindexlist.append(index)
    
print(f'*임베딩 시간 : {time.time()-start:.4f}')

In [ ]:
#-------------------------------------------------------------------------------------
# 4.쿼리 후 단어 MAX 스코어 합 구함
# => 쿼리 문장을 각 토큰별 임베딩값을 구하고, 이후 Faiss 문장 임베딩과 비교하여 가장 유사한 문장을 찾음
# => 3,000 문장 GPU 환경에서 약 4분 걸림.
#-------------------------------------------------------------------------------------
from myutils import df_sampling, sum_of_array_2d, index_of_list, get_vocab_bytag

QUERY_DIVIDE_LEN=3  # 쿼리를 몇개 토큰으로 나눠서 평균을 구할지 계수 설정

# 쿼리 샘플링함.
if query_num == 0:   # query_num = 0 이면 모든 쿼리(5533개)
    user_querys = df_questions['question'].values.tolist()
else:   # query_num > 0이면 해당 계수만큼 랜덤하게 샘플링하여 쿼리 목록을 만듬.
    df_questions = df_sampling(df=df_questions, num=query_num, seed=seed)
    user_querys = df_questions['answer'].values.tolist()
  
print(f'Query-----------------------------------------------------')
print(user_querys[0:3])

bi_predictions_list = []

for i, user_query in tqdm(enumerate(user_querys)):
    
    if i < 5:
        print(user_query)

    # 쿼리별 토큰 벡터를 구함 
    vectors_tmp = embed_text(model=bi_encoder, paragraphs=[user_query], token_embeddings=True, return_tensor=False)
    vectors_tmp2 = vectors_tmp[0].cpu().numpy().astype('float32') # tensor 토큰 벡터를 np.array로 변경
    vectors_tmp2 = vectors_tmp2[1:-1] # 맨앞에 [CLS]와 맨뒤에 [SEP] 토큰은 뺌
    vectors = divide_arr_avg(embed_arr=vectors_tmp2, divide_len=QUERY_DIVIDE_LEN) # QUERY_DIVIDE_LEN 만큼 자르면서 쿼리 토큰 평균을 구함
    
    # *cosine유사도 구할때는 반드시 normalize 처리함.
    if faiss_index_method == 0:
        faiss.normalize_L2(vectors)          
    
    max_values_list = []

    # 기존 contexts에 대한 faiss 문장 인덱스를 불러오면서, 단어 별루 쿼리 검색 해서 스코어를 구하고, 구한 단어별 스코어를 모두 더해서 총합을 구함.
    for count, index in enumerate(faissindexlist):
        
        # 각 faiss 문장 인덱스에서 문장단어 쿼리 검색 함 (distance, idx= numpy.array 타입임)
        distance, idx = index.search(np.array(vectors).astype("float32"), k=3) # 단어별 검색해서 3개 스코어를 얻어옴.

        # 예) distance = np.array([[2,3,4],[4,3,2]])
        # faiss_index_method=0 => cosine유사도 구할때는 max 값을 찾음, 1=>유클리드 거리 로 할때는 min 값을 찾음
        sum_value = sum_of_array_2d(array=distance, bmin=faiss_index_method)
           
        # 각 문장별 스코어 최대값을 저장해 둠.
        max_values_list.append(sum_value)

    # 문장별 스코어 최대값 리스트에서 최대값을 갖는 항목 search_k개 index만 출력 함.
    # faiss_index_method=0 => cosine유사도 구할때는 max 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [3,2,4])
    # faiss_index_method=1 => 유클리드 거리 로 할때는 min 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [1,0,4])
    indices = index_of_list(listdata=max_values_list, k=search_k, bmin=faiss_index_method)  
  
    # 예측검색결과 contextid값들을 리스트로 만듬.
    tmp_bi_predictions_list = []
    for indice in indices:
        tmp_bi_predictions_list.append(df_contexts["contextid"][indice])
    
    # 2D 예측검색결과 리스트에 추가 
    bi_predictions_list.append(tmp_bi_predictions_list)
    
print(bi_predictions_list[0:3])

In [ ]:
#--------------------------------------------------------------------------------------------------
# 5. MRR 계산
# => 정답 리스트[2,3,1,4] 과 예측검색리스트[[1,2,5,1],[3,4,2,1],[6,5,4,1], [2,3,4,1]]를 입력하여 MRR 스코어 구함
##--------------------------------------------------------------------------------------------------
from myutils import mean_reciprocal_rank

# 정답, 여기서는 contextid를 리스트로 만듬.
ground_truths_list = df_questions['contextid'].values.tolist()
print(f'gtlen:{len(ground_truths_list)}')
print(ground_truths_list[0:9])

logger.info(f'--------------------------------------------------------------------------')
logger.info('json_file:{}'.format(jsonfile))
logger.info(f'faiss 인덱싱 방식: {faiss_index_method}(0=코사인 유사도, 1=유클리드 거리)')
logger.info('search_k:{}/query_num:{}'.format(search_k, query_num))

# MRR 계산
bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, bi_predictions_list)

# BI-MRR 출력
logger.info(f'----------------------------------------------------------------------------')
logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
logger.info('*BI-MRR:{:.4f}'.format(bi_score))
logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:20]}')
    
# 검색 한 계슈
#logger.info(f'---------------------------------------------------------------------------')
search_count = 0
for item in bi_ranks:
    if item != 0:
        search_count += 1
    
logger.info('*검색률: {}/{}({:.2f}%)'.format(search_count, len(bi_ranks), (search_count/len(bi_ranks))*100))
logger.info(f'---------------------------------------------------------------------------')